Shoud we only predict the authorId or authorName shoudl be also predicted. Becuase Id and Name are interrelated. One unique name per one unique Id.

In [1]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'gensim'

In [39]:
train_data =open("Data/train.json")
train_i=json.load(train_data)
print(len(train_i))
df = pd.read_json("Data/train.json")
df.dtypes


12129


paperId       object
title         object
authorId       int64
authorName    object
abstract      object
year           int64
venue         object
dtype: object

In [40]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /Users/Ulyasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
#did this becuase I thought I can one-hot encode "venue" only. It should come as a first column if you one hot encode it only. 
cols = list(df.columns)
cols.reverse()
df_new=df[cols]
df_new.dtypes

venue         object
year           int64
abstract      object
authorName    object
authorId       int64
title         object
paperId       object
dtype: object

In [ ]:
#tokenizeing abstract and title columns. But what do we have to do after this?
df_new['tokenized_abstract'] = df_new.apply(lambda row: nltk.word_tokenize(row['abstract']), axis=1)
df_new['tokenized_title'] = df_new.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

In [27]:
df_new = df_new.drop(columns=["title","abstract"])

In [43]:
#METHOD 1. Not sure do we have to get dummies for PaperId (no dublicates, each PaperId is unique) and AuthorName? 
df_ohe = df_new
categorical_columns = ["venue"]
for col in categorical_columns:
    col_ohe = pd.get_dummies(df_new[col], prefix=col)
    df_ohe = pd.concat((df_ohe, col_ohe), axis=1).drop(col, axis=1)

#METHOD 2
#DOES NOT PERFORM AS PLANNED

categorical_cols = ["venue"] 

from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()

# apply le on categorical feature columns
df_new[categorical_cols] = df_new[categorical_cols].apply(lambda col: le.fit_transform(col))    
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

#One-hot-encode the categorical columns.
#Unfortunately outputs an array instead of dataframe.
array_hot_encoded = ohe.fit_transform(df_new[categorical_cols])

#Convert it to df
data_hot_encoded = pd.DataFrame(array_hot_encoded, index=df_new.index)

#Extract only the columns that didnt need to be encoded
data_other_cols = df_new.drop(columns=categorical_cols)

#Concatenate the two dataframes : 
data = pd.concat([data_hot_encoded, data_other_cols], axis=1)
data.columns.values

In [45]:
df_ohe.columns.get_loc("authorId")

2

In [ ]:
df_ohe.columns

In [12]:
X = df.iloc[lambda x: x.index != 2].values
y = df.iloc[:, 2].values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size= 0.4)
print(X_train.shape, X_val.shape, X_test.shape)